## Overhead
In order to know how fair our comparison is, we run all solvers repeatedly on the same, trivial instance.

In [4]:
!cat trivial.par

counter int R1;
synchronised {
  automaton value_0 {
    init s0;
    s0 -> s1 [32, 32] { R1 += 1 };
    accepting s1;
  };
};


In [153]:
%%sh
INVOCATIONS=$(perl -E 'say "trivial.par " x 30')

for backend in lazy nuxmv baseline;
do
  ~/catra/bin/catra solve-satisfy --backend ${backend} $INVOCATIONS > trivial-${backend}.log
done

In [154]:
df = pd.concat(pd.DataFrame(
    parse_log(f"trivial-{backend}.log"))
          .assign(backend = backend) for backend in backends)

NameError: name 'backends' is not defined

Now we can plot the values to see how they look over time. A clear result is that the first run is much slower for all backends, but less so for nuxmv. This is unsurprising, given that the other backends are running on a cold JVM, while nuxmv does most of the work in a compiled binary. We also see here that once the JVM is warm, the overhead for all back-ends normalise to something very similar, with a small, sub .1-second overhead for cold-starting nuxmv for each instance.

In [ ]:
_ = df.groupby("backend")["runtime"].plot(legend=True, 
                                      figsize=(10,6),
                                      xlabel="# iterations", 
                                      ylabel="Runtime (s) logscale",
                                      logy=True,
                                      title="Time spent solving the same trivial instance repeatedly",
                                      style='.')

Filtering out the outliers from the cold runs, we get the following:

In [ ]:
_ = df[df['runtime'] <= 0.1 ].groupby('backend')['runtime'].plot(figsize=(10, 6),
                                      legend=True, 
                                      xlabel="# iterations", 
                                      ylabel="Runtime (s) lin-scale",
                                      style='.',
                                      title="Time spent solving the same trivial instance repeatedly",
                                      logy=False)
plt.tight_layout()